In [48]:
import numpy as np 
import pandas as pd

from warnings import filterwarnings
filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 200)

In [49]:
df1 = pd.read_csv("movie_id_titles.csv")
df1.head(3)

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)


In [50]:
df2 = pd.read_csv("users.data")
df2.head(3)

,0\t50\t5\t881250949
0,0\t172\t5\t881250949
1,0\t133\t1\t881250949
2,196\t242\t3\t881250949


In [51]:
df2 = pd.read_csv("users.data",delimiter="\t",names=["user_id","item_id","rating","timestamp"])
df2.head(3)

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949


In [52]:
print(df1.shape)
print(df2.shape)

(1682, 2)
(100003, 4)


In [53]:
# df1 de filmlerin başlıkları mevcut
# df2 de filmlerin reytingleri mevcut
# bu iki bilgi bize lazım o yüzden item_id üzerinden bu iki datasetini birleştireceğiz.

df = pd.merge(df2,df1,on="item_id")
df.head(3)

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)


In [54]:
# kullanıcılar hangi filmlere kaç puanlık oy vermişler onu görüntüleyeceğiz.

tablo = df.pivot_table(index="user_id",columns="title",values="rating")
tablo.head(3)

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",8 1/2 (1963),8 Heads in a Duffel Bag (1997),8 Seconds (1994),A Chef in Love (1996),Above the Rim (1994),Absolute Power (1997),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Across the Sea of Time (1995),Addams Family Values (1993),Addicted to Love (1997),"Addiction, The (1995)","Adventures of Pinocchio, The (1996)","Adventures of Priscilla, Queen of the Desert, The (1994)","Adventures of Robin Hood, The (1938)","Affair to Remember, An (1957)","African Queen, The (1951)",Afterglow (1997),"Age of Innocence, The (1993)",Aiqing wansui (1994),Air Bud (1997),Air Force One (1997),"Air Up There, The (1994)",Airheads (1994),Akira (1988),Aladdin (1992),Aladdin and the King of Thieves (1996),Alaska (1996),Albino Alligator (1996),Alice in Wonderland (1951),Alien (1979),Alien 3 (1992),Alien: Resurrection (1997),Aliens (1986),All About Eve (1950),All Dogs Go to Heaven 2 (1996),All Over Me (1997),All Things Fair (1996),Alphaville (1965),Amadeus (1984),Amateur (1994),"Amazing Panda Adventure, The (1995)",American Buffalo (1996),American Dream (1990),"American President, The (1995)",American Strays (1996),"American Werewolf in London, An (1981)","American in Paris, An (1951)",Amistad (1997),Amityville 1992: It's About Time (1992),Amityville 3-D (1983),"Amityville Curse, The (1990)","Amityville Horror, The (1979)",Amityville II: The Possession (1982),Amityville: A New Generation (1993),Amityville: Dollhouse (1996),Amos & Andrew (1993),An Unforgettable Summer (1994),Anaconda (1997),Anastasia (1997),Andre (1994),Angel Baby (1995),Angel and the Badman (1947),Angel on My Shoulder (1946),Angela (1995),Angels and Insects (1995),Angels in the Outfield (1994),Angus (1995),Anna (1996),Anna Karenina (1997),Anne Frank Remembered (1995),Annie Hall (1977),Another Stakeout (1993),Antonia's Line (1995),Aparajito (1956),"Apartment, The (1960)",Apocalypse Now (1979),Apollo 13 (1995),"Apostle, The (1997)","Apple Dumpling Gang, The (1975)",April Fool's Day (1986),Apt Pupil (1998),"Aristocats, The (1970)",Army of Darkness (1993),Around the World in 80 Days (1956),"Arrival, The (1996)",Arsenic and Old Lace (1944),As Good As It Gets (1997),Assassins (1995),"Assignment, The (1997)","Associate, The (1996)",Audrey Rose (1977),August (1996),Austin Powers: International Man of Mystery (1997),"Awfully Big Adventure, An (1995)",Ayn Rand: A Sense of Life (1997),B*A*P*S (1997),B. Monkey (1998),Babe (1995),"Baby-Sitters Club, The (1995)",Babyfever (1994),"Babysitter, The (1995)",Back to the Future (1985),Backbeat (1993),Bad Boys (1995),Bad Company (1995),Bad Girls (1994),Bad Moon (1996),Bad Taste (1987),"Ballad of Narayama, The (Narayama Bushiko) (1958)",Balto (1995),Bananas (1971),"Band Wagon, The (1953)",Barb Wire (1996),Barbarella (1968),Barcelona (1994),Basic Instinct (1992),"Basketball Diaries, The (1995)",Basquiat (1996),Bastard Out of Carolina (1996),Batman & Robin (1997),Batman (1989),Batman Forever (1995),Batman Returns (1992),Baton Rouge (1988),Bean (1997),"Beans of Egypt, Maine, The (1994)",Beat the Devil (1954),"Beautician and the Beast, The (1997)",Beautiful Girls (1996),Beautiful Thing (1996),Beauty and the Beast (1991),Beavis and Butt-head Do America (1996),Bed of Roses (1996),Bedknobs and Broomsticks (1971),Before Sunrise (1995),Before and After (1996),Before the Rain (Pred dozhdot) (1994),Being Human (1993),Being There (1979),"Believers, The (1987)",Belle de jour (1967),Ben-Hur (1959),Benny & Joon (1993),Bent (1997),Best Men (1997),Best of the Best 3: No Turning Back (1995),Better Off Dead... (1985),"Beverly Hillbillies, The (1993)",Beverly Hills Cop III (1994),Beverly Hills Ninja (1997),"Bewegte Mann, Der (1994)",Beyond Bedlam (1993),Beyond Rangoon (1995),Bha

In [55]:
# Star Wars (1977) filmine kullanıcılar kaç puan vermiş görelim
starwars = tablo["Star Wars (1977)"]
starwars.head()

user_id
0   5.000
1   5.000
2   5.000
3     NaN
4   5.000
Name: Star Wars (1977), dtype: float64

In [56]:
# Star Wars (1977) filmine oy verenler diğer filmlere kaç puanlık oylama yapmışlar bunlar arasındaki ilişkiyi bulaacağız.

korelasyon = tablo.corrwith(starwars)
korelasyon

title
'Til There Was You (1997)                0.873
1-900 (1994)                            -0.645
101 Dalmatians (1996)                    0.211
12 Angry Men (1957)                      0.184
187 (1997)                               0.027
                                         ...  
Young Guns II (1990)                     0.229
Young Poisoner's Handbook, The (1995)   -0.007
Zeus and Roxanne (1997)                  0.818
unknown                                  0.723
Á köldum klaka (Cold Fever) (1994)         NaN
Length: 1664, dtype: float64

In [57]:
core = pd.DataFrame(korelasyon,columns=["correlation"])
core.dropna(inplace=True)

core =core.sort_values(by="correlation",ascending=False)
core.head(5)

,correlation
title,
Hollow Reed (1996),1.000
Commandments (1997),1.000
Cosi (1996),1.000
No Escape (1994),1.000
Stripes (1981),1.000


In [58]:
# her film için reytinglerin ortalamasını alalım

rating = pd.DataFrame(df.groupby("title")["rating"].mean())
rating = rating.sort_values(by="rating",ascending=False)
rating.head()

,rating
title,
They Made Me a Criminal (1939),5.000
Marlene Dietrich: Shadow and Light (1996),5.000
"Saint of Fort Washington, The (1993)",5.000
Someone Else's America (1995),5.000
Star Kid (1997),5.000


In [60]:
# oy sayısını bulalım. film isimlerine göre grupladım ve kaç kişi oy vermiş saydırdım

rating["oysayısı"] = pd.DataFrame(df.groupby("title")["rating"].count())
rating = rating.sort_values(by="oysayısı",ascending=False)
rating.head()

,rating,oysayısı
title,,
Star Wars (1977),4.360,584
Contact (1997),3.804,509
Fargo (1996),4.156,508
Return of the Jedi (1983),4.008,507
Liar Liar (1997),3.157,485


In [61]:
# yukarıda hesaplamış olduğumuz korelasyona oy sayısını da ekleyelim
core = core.join(rating["oysayısı"])
core.head()

,correlation,oysayısı
title,,
Hollow Reed (1996),1.000,6
Commandments (1997),1.000,3
Cosi (1996),1.000,4
No Escape (1994),1.000,5
Stripes (1981),1.000,5


In [64]:
# oy sayısı düşük olanları çıkaralım
core = core[core["oysayısı"]>100].sort_values(by="correlation",ascending=False)
core.head()

,correlation,oysayısı
title,,
Star Wars (1977),1.000,584
"Empire Strikes Back, The (1980)",0.748,368
Return of the Jedi (1983),0.673,507
Raiders of the Lost Ark (1981),0.536,420
Austin Powers: International Man of Mystery (1997),0.377,130
